In [1]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.python.keras.preprocessing.text import Tokenizer

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_words = 5000
batch_size = 128
epochs = 5

Getting data:

In [3]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']
categories = None
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)
from pprint import pprint
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [4]:
newsgroups_train.filenames.shape

(11314,)

In [5]:
newsgroups_train.target.shape

(11314,)

In [6]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Converting text to vectors:

In [7]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(newsgroups_train["data"])
x_train = tokenizer.texts_to_matrix(newsgroups_train["data"], mode='binary')
x_test = tokenizer.texts_to_matrix(newsgroups_test["data"], mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (11314, 5000)
x_test shape: (7532, 5000)


In [8]:
num_classes = np.max(newsgroups_train["target"]) + 1
print(num_classes, 'classes')

20 classes


Convert class vector to binary class matrix (for use with categorical_crossentropy):

In [9]:
y_train = keras.utils.to_categorical(newsgroups_train["target"], num_classes)
y_test = keras.utils.to_categorical(newsgroups_test["target"], num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (11314, 20)
y_test shape: (7532, 20)


Building model functionally:

In [10]:
a = Input(shape=(max_words,))
b = Dense(512)(a)
b = Activation('relu')(b)
b = Dropout(0.5)(b)
b = Dense(num_classes)(b)
b = Activation('softmax')(b)
model = Model(inputs=a, outputs=b)

In [11]:
model.layers

In [12]:
print(model.to_yaml())

backend: tensorflow
class_name: Model
config:
  input_layers:
  - [input_1, 0, 0]
  layers:
  - class_name: InputLayer
    config:
      batch_input_shape: !!python/tuple [null, 5000]
      dtype: float32
      name: input_1
      sparse: false
    inbound_nodes: []
    name: input_1
  - class_name: Dense
    config:
      activation: linear
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {dtype: float32}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: VarianceScaling
        config: {distribution: uniform, dtype: float32, mode: fan_avg, scale: 1.0,
          seed: null}
      kernel_regularizer: null
      name: dense
      trainable: true
      units: 512
      use_bias: true
    inbound_nodes:
    - - - input_1
        - 0
        - 0
        - {}
    name: dense
  - class_name: Activation
    config: {activation: relu, dtype: f

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
estimator = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\aserg\\AppData\\Local\\Temp\\tmpxv9y22qr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000128C0349E10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    num_epochs=epochs,
    shuffle=False)
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\aserg\AppData\Local\Temp\tmpxv9y22qr\keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\aserg\AppData\Local\Temp\tmpxv9y22qr\model.ckpt.
INFO:tensorflow:loss = 3.0597196, step = 1
INFO:tensorflow:global_step/sec: 8.07114
INFO:tensorflow:loss = 0.4679227, step = 101 (12.390 sec)
INFO:tensorflow:global_step/sec: 9.02708
INFO:tensorflow:loss = 0.20524035, step = 201 (11.078 sec)
INFO:tensorflow:global_step/sec: 9.60992
INFO:tensorflow:loss = 0.15861331, step = 301 (10.422 sec)
INFO:tensorflow:global_step/sec: 8.91393
INFO:tensorflow:loss = 0.097175226, step = 401 (11.218 sec)
INFO:tensorflow:Saving checkpoints for 443 into C:\Users\aserg\AppData\Local\Temp\tmpxv9y22qr\model.ckpt.
INFO:

In [16]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=x_test,
    y=y_test,
    batch_size=batch_size,
    shuffle=False)
score = estimator.evaluate(input_fn=test_input_fn)
print('\n')
print('Score:', score)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-22-15:57:10
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\aserg\AppData\Local\Temp\tmpxv9y22qr\model.ckpt-443
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-22-15:57:11
INFO:tensorflow:Saving dict for global step 443: accuracy = 0.80992526, global_step = 443, loss = 0.7120646
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 443: C:\Users\aserg\AppData\Local\Temp\tmpxv9y22qr\model.ckpt-443


Score: {'accuracy': 0.80992526, 'loss': 0.7120646, 'global_step': 443}
